In [1]:
path = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'

In [2]:
!pip install modin[dask]
!pip install polars
!pip install pyarrow

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [3]:
!wget -r -nc https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv

In [4]:
import polars as pl
import pyarrow as pa
import pandas as pd
import dask
import dask.dataframe as dd
import modin.pandas as md
import pyarrow.compute as pc

In [5]:
pd.read_csv('Global_Mobility_Report.csv', nrows=3)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0,4,5,0,2,1
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1,4,4,1,2,1
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1,1,5,1,2,1


In [6]:
pd.read_csv('Global_Mobility_Report.csv', nrows=3).columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [7]:
columns = ['country_region_code', 'country_region', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']

In [8]:
# dd.read_csv('Global_Mobility_Report.csv').to_parquet('mobility.parquet',compression='snappy',engine='pyarrow', low_memory=False)
(dd.read_csv('Global_Mobility_Report.csv',
            assume_missing=True, 
            usecols= columns,
            parse_dates=['date'])
   .head()
)

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [9]:
(dd.read_csv('Global_Mobility_Report.csv',
            assume_missing=True, 
            usecols= columns,
            parse_dates=['date'])
).to_parquet('mobility.parquet',compression='snappy',engine='pyarrow')


"(dd.read_csv('Global_Mobility_Report.csv',\n            assume_missing=True, \n            usecols= columns,\n            parse_dates=['date'])\n).to_parquet('mobility.parquet',compression='snappy',engine='pyarrow')\n"

In [10]:
!ls

Global_Mobility_Report.csv  dask-worker-space  tutorial-env
Test.ipynb		    mobility.parquet   www.gstatic.com


In [11]:
%time dddf = dd.read_parquet('mobility.parquet', engine='pyarrow')

CPU times: user 20.6 ms, sys: 6.53 ms, total: 27.2 ms
Wall time: 31.4 ms


In [12]:
from distributed import Client
client = Client()

/usr/lib/python3.9/contextlib.py:126: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


In [13]:
%time mddf = md.read_parquet('mobility.parquet', engine='pyarrow')

CPU times: user 583 ms, sys: 374 ms, total: 956 ms
Wall time: 3.78 s


In [14]:
%time pdf = pd.read_parquet('mobility.parquet', engine='pyarrow')

CPU times: user 1.92 s, sys: 1.24 s, total: 3.16 s
Wall time: 2.87 s


In [15]:
%time pldf = pl.read_parquet('mobility.parquet', use_pyarrow=True)

CPU times: user 1.42 s, sys: 1.23 s, total: 2.65 s
Wall time: 2.35 s


In [16]:
%time padf = pa.parquet.read_table('mobility.parquet')

CPU times: user 1.15 s, sys: 486 ms, total: 1.63 s
Wall time: 1.41 s


## Read Shape

In [17]:
%time dddf.shape[0].compute()

CPU times: user 624 ms, sys: 186 ms, total: 810 ms
Wall time: 5.51 s


10806198

In [18]:
%time pdf.shape

CPU times: user 13 µs, sys: 7 µs, total: 20 µs
Wall time: 26.2 µs


(10806198, 9)

In [19]:
%time mddf.shape

CPU times: user 34 µs, sys: 18 µs, total: 52 µs
Wall time: 56.7 µs


(10806198, 9)

In [20]:
%time pldf.shape

CPU times: user 25 µs, sys: 13 µs, total: 38 µs
Wall time: 42.9 µs


(10806198, 10)

In [21]:
%time padf.shape

CPU times: user 8 µs, sys: 5 µs, total: 13 µs
Wall time: 17.4 µs


(10806198, 10)

Pyarrow is the fastest. Not sure why Modin is slower than even Pandas.

## Country Region Count

In [22]:
%time dddf['country_region'].value_counts().nlargest(10).compute()

CPU times: user 77.2 ms, sys: 19 ms, total: 96.2 ms
Wall time: 676 ms


United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [23]:
%time pdf['country_region'].value_counts().nlargest(10)

CPU times: user 475 ms, sys: 0 ns, total: 475 ms
Wall time: 464 ms


United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [24]:
%time mddf['country_region'].value_counts().nlargest(10)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


CPU times: user 653 ms, sys: 59.7 ms, total: 713 ms
Wall time: 1.6 s


United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [25]:
%time pldf.groupby('country_region').count().sort(by = 'count', reverse=True).head(10)

CPU times: user 788 ms, sys: 303 ms, total: 1.09 s
Wall time: 299 ms


country_region,count
str,u32
"""United States""",2312412
"""Brazil""",1808620
"""India""",598225
"""Turkey""",493512
"""Argentina""",378917
"""United Kingdom...",374096
"""Poland""",346884
"""Netherlands""",319961
"""Colombia""",286514


In [26]:
%time mddf.groupby('country_region')['country_region'].count().sort_values(ascending=False).head(10)

CPU times: user 669 ms, sys: 104 ms, total: 773 ms
Wall time: 3.45 s


country_region
United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [27]:
%time pa.TableGroupBy(padf,'country_region').aggregate([('country_region', "count")]).to_pandas()[["country_region","country_region_count"]]

CPU times: user 493 ms, sys: 14.5 ms, total: 508 ms
Wall time: 479 ms


,country_region,country_region_count
0,United Arab Emirates,7169
1,Afghanistan,1704
2,Antigua and Barbuda,3627
3,Angola,4136
4,Argentina,378917
...,...,...
130,Panama,1794
131,Peru,117911
132,Papua New Guinea,894
133,Philippines,23322


In [28]:
%time  (padf['country_region'].value_counts()).to_pandas()

CPU times: user 245 ms, sys: 0 ns, total: 245 ms
Wall time: 226 ms


0      {'values': 'United Arab Emirates', 'counts': 7...
1              {'values': 'Afghanistan', 'counts': 1704}
2      {'values': 'Antigua and Barbuda', 'counts': 3627}
3                   {'values': 'Angola', 'counts': 4136}
4              {'values': 'Argentina', 'counts': 378917}
                             ...                        
130                 {'values': 'Panama', 'counts': 1794}
131                 {'values': 'Peru', 'counts': 117911}
132        {'values': 'Papua New Guinea', 'counts': 894}
133           {'values': 'Philippines', 'counts': 23322}
134              {'values': 'Pakistan', 'counts': 18787}
Length: 135, dtype: object

## GROUPBY OPERATION

In [29]:
%time dddf.groupby(['country_region',dddf.date.dt.month_name() ])['grocery_and_pharmacy_percent_change_from_baseline'].mean().compute()

CPU times: user 761 ms, sys: 167 ms, total: 928 ms
Wall time: 7.24 s


country_region  date     
Afghanistan     April        59.950000
                August       56.489362
                December     64.064516
                February     54.802817
                January      60.564516
                               ...    
Zimbabwe        March        43.189616
                May          47.511468
                November     47.378092
                October      46.393220
                September    40.555133
Name: grocery_and_pharmacy_percent_change_from_baseline, Length: 1620, dtype: float64

In [30]:
%time pdf.groupby(['country_region',pdf.date.dt.month_name() ])['grocery_and_pharmacy_percent_change_from_baseline'].mean()

CPU times: user 5.99 s, sys: 3.28 s, total: 9.27 s
Wall time: 8.82 s


country_region  date     
Afghanistan     April        59.950000
                August       56.489362
                December     64.064516
                February     54.802817
                January      60.564516
                               ...    
Zimbabwe        March        43.189616
                May          47.511468
                November     47.378092
                October      46.393220
                September    40.555133
Name: grocery_and_pharmacy_percent_change_from_baseline, Length: 1620, dtype: float64

In [31]:
mddf.groupby(['country_region',mddf.date.dt.month_name() ])['grocery_and_pharmacy_percent_change_from_baseline'].mean()

NotImplementedError: Column lookups on GroupBy with arbitrary Series in by is not yet supported.

In [32]:
%%time
(mddf[['country_region','date', 'grocery_and_pharmacy_percent_change_from_baseline']]
 .assign(month = mddf.date.dt.month_name())
 .drop('date',axis=1)
).groupby(['country_region','month'])['grocery_and_pharmacy_percent_change_from_baseline'].mean()

CPU times: user 1.97 s, sys: 472 ms, total: 2.44 s
Wall time: 20.9 s


country_region  month    
Afghanistan     April        59.950000
                August       56.489362
                December     64.064516
                February     54.802817
                January      60.564516
                               ...    
Zimbabwe        March        43.189616
                May          47.511468
                November     47.378092
                October      46.393220
                September    40.555133
Name: grocery_and_pharmacy_percent_change_from_baseline, Length: 1620, dtype: float64

In [33]:
month_names = {"1":"January", "2":"February", "3":"March","4":"April","5":"May","6":"June","7":"July","8":"August","9":"September","10":"October","11":"November","12":"December"}
month_names

{'1': 'January',
 '2': 'February',
 '3': 'March',
 '4': 'April',
 '5': 'May',
 '6': 'June',
 '7': 'July',
 '8': 'August',
 '9': 'September',
 '10': 'October',
 '11': 'November',
 '12': 'December'}

In [34]:
%%time
(
(pldf[['country_region','date', 'grocery_and_pharmacy_percent_change_from_baseline']]
 .with_column(pldf['date'].dt.month()
              .cast(str)
              .apply(lambda month:month_names.get(month)))
 
)
    .groupby(['country_region','date']).agg([pl.col('grocery_and_pharmacy_percent_change_from_baseline').mean().alias("mean")])
    .sort(by=["country_region","date"])
)

CPU times: user 7.36 s, sys: 2.07 s, total: 9.43 s
Wall time: 7.79 s


country_region,date,mean
str,str,f64
"""Afghanistan""","""April""",59.95
"""Afghanistan""","""August""",56.489362
"""Afghanistan""","""December""",64.064516
"""Afghanistan""","""February""",54.802817
"""Afghanistan""","""January""",60.564516
"""Afghanistan""","""July""",85.522222
"""Afghanistan""","""June""",118.475
"""Afghanistan""","""March""",70.295699
"""Afghanistan""","""May""",87.64375


In [35]:
%%time
padf2 = padf.append_column("month",pc.month(padf['date']))
(pa.TableGroupBy(padf2,['country_region', "month"])
 .aggregate([("grocery_and_pharmacy_percent_change_from_baseline", "mean")])
).to_pandas()[['country_region','month','grocery_and_pharmacy_percent_change_from_baseline_mean']].sort_values(by = ['country_region','month'])
    

CPU times: user 820 ms, sys: 77.4 ms, total: 898 ms
Wall time: 831 ms


,country_region,month,grocery_and_pharmacy_percent_change_from_baseline_mean
22,Afghanistan,1,60.564516
12,Afghanistan,2,54.802817
13,Afghanistan,3,70.295699
14,Afghanistan,4,59.950000
15,Afghanistan,5,87.643750
...,...,...,...
618,Zimbabwe,8,25.034221
619,Zimbabwe,9,40.555133
620,Zimbabwe,10,46.393220
621,Zimbabwe,11,47.378092


Pyarrow is crazy fast but doesnsn't follow pandas API, requires passing df to pandas. Polars is fast and almost similar to pandas. Modin is slower than pandas. 